In [3]:
import pandas as pd 
import numpy as np 
import scipy as sp 
import matplotlib.pyplot as plt 

Reading and Analysis of whole data

In [111]:
data = pd.read_csv('data.csv')

In [5]:
d1.shape

(30697, 28)

In [6]:
d1.columns

Index(['Unnamed: 0', 'match_event_id', 'location_x', 'location_y',
       'remaining_min', 'power_of_shot', 'knockout_match', 'game_season',
       'remaining_sec', 'distance_of_shot', 'is_goal', 'area_of_shot',
       'shot_basics', 'range_of_shot', 'team_name', 'date_of_game',
       'home/away', 'shot_id_number', 'lat/lng', 'type_of_shot',
       'type_of_combined_shot', 'match_id', 'team_id', 'remaining_min.1',
       'power_of_shot.1', 'knockout_match.1', 'remaining_sec.1',
       'distance_of_shot.1'],
      dtype='object')

In [7]:
#dropping column team name, team id because it's same throughout and seconds don't hold much importance in goal detection
d1 = d1.drop(['team_name','team_id','shot_id_number','remaining_sec','remaining_sec.1'],axis=1)

In [8]:
#filling in the missing values in shot_id_number column 
d1['Unnamed: 0'] += 1
d1 = d1.rename(columns={'Unnamed: 0': 'shot_id_number'})

Distance_of_shot Analysis

In [9]:
#to see whether the values are same or not 
d2 = d1[['distance_of_shot','distance_of_shot.1']]
d2 = d2.rename(columns={'distance_of_shot.1':'distance_of_shot_1'})
d2.sample(7)

,distance_of_shot,distance_of_shot_1
4895,40.0,40.000
357,28.0,28.000
24473,29.0,29.000
16743,37.0,89.728
6255,20.0,20.000
4032,46.0,86.728
11091,39.0,39.000


In [10]:
#filling na values and finding average of the 2 cols
d2.distance_of_shot.fillna(d2.distance_of_shot_1, inplace=True)
d2.distance_of_shot_1.fillna(d2.distance_of_shot, inplace=True)
d2['distance_of_shot_avg']=d2.mean(axis=1)
d2.head()

,distance_of_shot,distance_of_shot_1,distance_of_shot_avg
0,38.0,38.0,38.0
1,35.0,35.0,35.0
2,36.0,54.4,45.2
3,42.0,42.0,42.0
4,20.0,20.0,20.0


In [11]:
#concatinating d1 and d2 
d1 = d1.drop(['distance_of_shot','distance_of_shot.1'],axis=1)
d3 = pd.concat([d1, d2], axis=1)
d3.sample(2)

,shot_id_number,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,is_goal,area_of_shot,...,lat/lng,type_of_shot,type_of_combined_shot,match_id,remaining_min.1,power_of_shot.1,knockout_match.1,distance_of_shot,distance_of_shot_1,distance_of_shot_avg
1249,1250,353.0,0.0,0.0,2.0,3.0,0.0,2000-01,1.0,Center(C),...,"42.982923, -71.446094",NaN,shot - 4,20000860,2.0,3.0,0.0,20.0,20.0,20.0
19973,19974,377.0,-4.0,11.0,2.0,3.0,NaN,2012-13,1.0,Center(C),...,"37.754130, -122.437947",NaN,shot - 1,21200398,2.0,3.0,0.0,21.0,21.0,21.0


In [12]:
#filling in the na values in distance of shot avg with the help of range of shot 
n1 = d3['distance_of_shot_avg'].isnull()
n1[n1==True].count()

97

In [13]:
r = d3.groupby('range_of_shot')['distance_of_shot_avg'].agg('mean')
r

range_of_shot
16-24 ft.          40.742291
24+ ft.            46.707029
8-16 ft.           34.827985
Back Court Shot    72.619210
Less Than 8 ft.    25.586323
Name: distance_of_shot_avg, dtype: float64

In [14]:
d3.distance_of_shot_avg.fillna(d3.range_of_shot, inplace=True)

In [15]:
for i in d3.index:
  val = d3.get_value(i,'distance_of_shot_avg')
  if(val=='Less Than 8 ft.'):
    d3.set_value(i,'distance_of_shot_avg',25.5863)
  elif(val=='16-24 ft.'):
    d3.set_value(i,'distance_of_shot_avg',40.74)
  elif(val=='24+ ft.'):
    d3.set_value(i,'distance_of_shot_avg',46.707029)
  elif(val=='8-16 ft.'):
    d3.set_value(i,'distance_of_shot_avg',34.8279)
  elif(val=='Back Court Shot'):
    d3.set_value(i,'distance_of_shot_avg',72.619)

In [16]:
d3.columns

Index(['shot_id_number', 'match_event_id', 'location_x', 'location_y',
       'remaining_min', 'power_of_shot', 'knockout_match', 'game_season',
       'is_goal', 'area_of_shot', 'shot_basics', 'range_of_shot',
       'date_of_game', 'home/away', 'lat/lng', 'type_of_shot',
       'type_of_combined_shot', 'match_id', 'remaining_min.1',
       'power_of_shot.1', 'knockout_match.1', 'distance_of_shot',
       'distance_of_shot_1', 'distance_of_shot_avg'],
      dtype='object')

In [17]:
d3['distance_of_shot_avg'].fillna(36, inplace=True)

Remaining_min Analysis

In [18]:
#to see whether the values are same or not 
d4 = d1[['remaining_min','remaining_min.1']]
d4 = d4.rename(columns={'remaining_min.1':'remaining_min_1'})
d4.head()

,remaining_min,remaining_min_1
0,10.0,10.00
1,10.0,10.00
2,7.0,92.64
3,6.0,NaN
4,NaN,42.64


In [19]:
d4.describe()

,remaining_min,remaining_min_1
count,29135.000000,29162.000000
mean,4.883233,18.204615
std,3.452533,29.416973
min,0.000000,0.000000
25%,2.000000,3.000000
50%,5.000000,6.000000
75%,8.000000,11.000000
max,11.000000,128.761600


In [20]:
#filling na values in remaining_min_1 with remaining_min 
d4.remaining_min_1.fillna(d4.remaining_min, inplace=True)
d4= d4.drop('remaining_min',axis=1)

In [21]:
#concatinating these the average now d5 is latest 
d5 = pd.concat([d3, d4], axis=1)
d5 = d5.drop(['remaining_min','remaining_min.1'],axis=1)
d5.columns

Index(['shot_id_number', 'match_event_id', 'location_x', 'location_y',
       'power_of_shot', 'knockout_match', 'game_season', 'is_goal',
       'area_of_shot', 'shot_basics', 'range_of_shot', 'date_of_game',
       'home/away', 'lat/lng', 'type_of_shot', 'type_of_combined_shot',
       'match_id', 'power_of_shot.1', 'knockout_match.1', 'distance_of_shot',
       'distance_of_shot_1', 'distance_of_shot_avg', 'remaining_min_1'],
      dtype='object')

In [22]:
d5['remaining_min_1'].describe()

count    30616.000000
mean        17.575091
std         28.857798
min          0.000000
25%          3.000000
50%          6.000000
75%         10.000000
max        128.761600
Name: remaining_min_1, dtype: float64

In [23]:
d5['remaining_min_1'].fillna(17,inplace=True)

knockout match Analysis

In [24]:
#selecting the appropriate column for knockout match 
d5 = d5.rename(columns={'knockout_match.1':'knockout_match_1'})
d5.knockout_match.fillna(d5.knockout_match_1, inplace=True)
d5 = d5.drop('knockout_match_1',axis=1)

In [25]:
d5['knockout_match'].fillna("U", inplace = True)

In [26]:
d5['knockout_match'] = d5['knockout_match'].replace(to_replace ='U', value = '0.0', regex = True) 
d5['knockout_match'] = pd.to_numeric(d5['knockout_match'])

for i in d5.index:
  val = d5.get_value(i,'knockout_match')
  if(val>1.0):
    d5.set_value(i,'knockout_match',1.0)

In [27]:
d5['knockout_match'].unique()

array([0., 1.])

power of shot Analysis

In [28]:
#power of shot has 1 to 7 makes more sense
d5 = d5.rename(columns={'power_of_shot.1':'power_of_shot_1'})
d5.power_of_shot.fillna(d5.power_of_shot_1, inplace=True)
d5 = d5.drop('power_of_shot_1',axis=1)

In [29]:
d5['power_of_shot'].describe()

count    30627.000000
mean         3.108937
std          6.761758
min          1.000000
25%          1.000000
50%          3.000000
75%          4.000000
max        118.360000
Name: power_of_shot, dtype: float64

In [30]:
d5.loc[d5['power_of_shot'] > 7, 'power_of_shot'] = np.nan

In [31]:
d5['power_of_shot'].unique()

array([ 1.,  2.,  3.,  4., nan,  5.,  6.,  7.])

In [32]:
d5['power_of_shot'].mode()

0    3.0
dtype: float64

In [33]:
d5['power_of_shot'].fillna(3, inplace = True)
d5['power_of_shot'].unique()

array([1., 2., 3., 4., 5., 6., 7.])

shot basics and area of shot Analysis

In [34]:
n2=d5['area_of_shot'].isnull()
n2[n2==True].count()

1502

In [35]:
#because the value of na is large shouldn't be replaced by mode
d5['area_of_shot'].fillna("U", inplace = True)
d5['shot_basics'].fillna("U", inplace = True)

In [36]:
d5 = pd.get_dummies(d5,prefix_sep="_",columns=["area_of_shot","shot_basics"])
d5.shape

(30697, 34)

In [37]:
d5.dtypes

shot_id_number                          int64
match_event_id                        float64
location_x                            float64
location_y                            float64
power_of_shot                         float64
knockout_match                        float64
game_season                            object
is_goal                               float64
range_of_shot                          object
date_of_game                           object
home/away                              object
lat/lng                                object
type_of_shot                           object
type_of_combined_shot                  object
match_id                                int64
distance_of_shot                      float64
distance_of_shot_1                    float64
distance_of_shot_avg                  float64
remaining_min_1                       float64
area_of_shot_Center(C)                  uint8
area_of_shot_Left Side Center(LC)       uint8
area_of_shot_Left Side(L)         

game_season Analysis

In [38]:
d5['game_season'].unique()
d5['game_season'].fillna(method='ffill', inplace=True)

type of shot Analysis

In [39]:
#filling 0 in place of na 
d5['type_of_combined_shot'].fillna(0, inplace=True)
d5['type_of_shot'].fillna(0, inplace = True)

In [143]:
d5.to_csv('first.csv',index=False)

location x and y Analysis

In [144]:
d5 = pd.read_csv('first.csv')
d6 = d5.drop('location_x',axis=1)
d6 = d6.drop('location_y',axis=1)

home/away and lat/lng analysis

In [145]:
d6['home/away'] = d6.groupby(['match_id'])['home/away'].ffill()
d6['home/away'] = d6.groupby(['match_id'])['home/away'].bfill()

In [146]:
d6['lat/lng'].fillna(d6['home/away'],inplace=True)
d6['lat/lng'] = d6['lat/lng'].replace(to_replace ='@', value = 0 , regex = True) 
d6['lat/lng'] = d6['lat/lng'].replace(to_replace ='vs', value = 1 , regex = True) 

In [147]:
d6.loc[(d6['lat/lng'] != '42.982923, -71.446094') & (d6['lat/lng'] != 1) &  (d6['lat/lng'].notnull()), 'lat/lng'] = 0
d6.loc[d6['lat/lng'] == '42.982923, -71.446094','lat/lng'] = 1
d6['lat/lng'].unique()

array([0, 1], dtype=object)

In [148]:
#to get name of opposition teams
d6['home/away']=d6['home/away'].astype("str").apply(lambda x: x.lstrip("MANU"))
d6['home/away']=d6['home/away'].astype("str").apply(lambda x: x.lstrip("vs. "))
d6['home/away']=d6['home/away'].astype("str").apply(lambda x: x.lstrip("@ "))

In [149]:
d6 = d6.drop(['range_of_shot','distance_of_shot','distance_of_shot_1'], axis=1)
d6 = d6.drop(['match_event_id','date_of_game'],axis=1)

In [150]:
#d7 = d6['is_goal']

In [151]:
#to check if we got rid of all na values
c= d6.isnull()
c[c==True].count()

shot_id_number                           0
power_of_shot                            0
knockout_match                           0
game_season                              0
is_goal                               6268
home/away                                0
lat/lng                                  0
type_of_shot                             0
type_of_combined_shot                    0
match_id                                 0
distance_of_shot_avg                     0
remaining_min_1                          0
area_of_shot_Center(C)                   0
area_of_shot_Left Side Center(LC)        0
area_of_shot_Left Side(L)                0
area_of_shot_Mid Ground(MG)              0
area_of_shot_Right Side Center(RC)       0
area_of_shot_Right Side(R)               0
area_of_shot_U                           0
shot_basics_Goal Area                    0
shot_basics_Goal Line                    0
shot_basics_Left Corner                  0
shot_basics_Mid Ground Line              0
shot_basics

In [152]:
d6['is_goal'].fillna(5, inplace=True)
d6['is_goal'] = pd.to_numeric(d6['is_goal'])

In [153]:
d6.to_csv('second.csv',index=False)

handling categorical data

In [154]:
f1 = pd.read_csv('second.csv')

In [155]:
f1.shape

(30697, 27)

In [156]:
f2 = f1.groupby(['home/away'])['is_goal'].agg('sum').reset_index().sort_values(by='is_goal',ascending=False)
f2 = pd.DataFrame(f2)
f2.head()

,home/away,is_goal
31,SAS,2777.0
8,DEN,2374.0
11,HOU,2270.0
29,SAC,2182.0
28,POR,2133.0


In [157]:
f3 = f1.groupby(['match_id'])['is_goal'].agg('sum').reset_index().sort_values(by='is_goal',ascending=False)
f3 = pd.DataFrame(f3)
f3.head()

,match_id,is_goal
466,20600431,100.0
199,20200769,84.0
102,20100525,79.0
553,20700553,77.0
197,20200743,69.0


In [158]:
#f4 = f1.groupby(['area_of_shot'])['is_goal'].agg('sum').reset_index().sort_values(by='is_goal',ascending=False)
#f4 = pd.DataFrame(f4)
#f4

In [159]:
#f5 = f1.groupby(['shot_basics'])['is_goal'].agg('sum').reset_index().sort_values(by='is_goal',ascending=False)
#f5 = pd.DataFrame(f5)
#f5

In [160]:
f6 = f1.groupby(['game_season'])['is_goal'].agg('sum').reset_index().sort_values(by='is_goal',ascending=False)
f6 = pd.DataFrame(f6)
f6.head()

,game_season,is_goal
9,2005-06,3322.0
12,2008-09,3215.0
6,2002-03,3163.0
11,2007-08,2946.0
5,2001-02,2711.0


In [161]:
f7 = f1.groupby(['type_of_shot'])['is_goal'].agg('sum').reset_index().sort_values(by='is_goal',ascending=False)
f7 = pd.DataFrame(f7)
#f7[type_of_shot].head()

In [162]:
f8 = f1.groupby(['type_of_combined_shot'])['is_goal'].agg('sum').reset_index().sort_values(by='is_goal',ascending=False)
f8 = pd.DataFrame(f8)
f8['type_of_combined_shot'].unique()

array(['0', 'shot - 3', 'shot - 4', 'shot - 1', 'shot - 0', 'shot - 2',
       'shot - 5'], dtype=object)

replacing the string values with numeric values using merge

In [124]:
tocs = ['0', 'shot - 3', 'shot - 4', 'shot - 1', 'shot - 0', 'shot - 2',
       'shot - 5']

In [125]:
m1 = pd.merge(f1, f2, on ='home/away')
m1 = m1.rename(columns={'is_goal_y':'home/away_n'})
#m1.columns 

In [126]:
m2 = pd.merge(m1, f3, on ='match_id')
m2 = m2.rename(columns={'is_goal':'match_id_n'})
#m2.columns

In [127]:
#m3 = pd.merge(m2, f4, on ='area_of_shot')
#m3 = m3.rename(columns={'is_goal':'area_of_shot_n'})
#m3.columns

In [65]:
#m4 = pd.merge(m3, f5, on ='shot_basics')
#m4 = m4.rename(columns={'is_goal':'shot_basics_n'})
#m4.columns

In [128]:
m5 = pd.merge(m2, f6, on ='game_season')
m5 = m5.rename(columns={'is_goal':'game_season_n'})
#m5.columns

In [129]:
m6 = pd.merge(m5, f7, on ='type_of_shot')
m6 = m6.rename(columns={'is_goal':'type_of_shot_n'})
#m6.columns

In [130]:
m7 = pd.merge(m6, f8, on ='type_of_combined_shot')
m7 = m7.rename(columns={'is_goal':'type_of_combined_shot_n'})
m7.columns

Index(['shot_id_number', 'power_of_shot', 'knockout_match', 'game_season',
       'is_goal_x', 'home/away', 'lat/lng', 'type_of_shot',
       'type_of_combined_shot', 'match_id', 'distance_of_shot_avg',
       'remaining_min_1', 'area_of_shot_Center(C)',
       'area_of_shot_Left Side Center(LC)', 'area_of_shot_Left Side(L)',
       'area_of_shot_Mid Ground(MG)', 'area_of_shot_Right Side Center(RC)',
       'area_of_shot_Right Side(R)', 'area_of_shot_U', 'shot_basics_Goal Area',
       'shot_basics_Goal Line', 'shot_basics_Left Corner',
       'shot_basics_Mid Ground Line', 'shot_basics_Mid Range',
       'shot_basics_Penalty Spot', 'shot_basics_Right Corner', 'shot_basics_U',
       'home/away_n', 'match_id_n', 'game_season_n', 'type_of_shot_n',
       'type_of_combined_shot_n'],
      dtype='object')

In [131]:
m8 = m7.drop(['home/away','match_id', 'game_season'],axis=1)
m8 = m8.drop(['type_of_shot','type_of_combined_shot'],axis=1)
m8.columns

Index(['shot_id_number', 'power_of_shot', 'knockout_match', 'is_goal_x',
       'lat/lng', 'distance_of_shot_avg', 'remaining_min_1',
       'area_of_shot_Center(C)', 'area_of_shot_Left Side Center(LC)',
       'area_of_shot_Left Side(L)', 'area_of_shot_Mid Ground(MG)',
       'area_of_shot_Right Side Center(RC)', 'area_of_shot_Right Side(R)',
       'area_of_shot_U', 'shot_basics_Goal Area', 'shot_basics_Goal Line',
       'shot_basics_Left Corner', 'shot_basics_Mid Ground Line',
       'shot_basics_Mid Range', 'shot_basics_Penalty Spot',
       'shot_basics_Right Corner', 'shot_basics_U', 'home/away_n',
       'match_id_n', 'game_season_n', 'type_of_shot_n',
       'type_of_combined_shot_n'],
      dtype='object')

In [132]:
m8 = m8.rename(columns={'lat/lng':'home/away', 'home/away_n':'opposing_team'})

In [133]:
m8['is_goal_x'].unique()

array([0., 1.])

In [134]:
g=m8['is_goal_x']==5
g[g==True].count()

0

NameError: name 'test' is not defined

machine learning model

In [84]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [85]:
result_dict = {}

In [86]:
def summarize_classification(y_test, y_pred):
    
    acc = accuracy_score(y_test, y_pred, normalize=True)
    num_acc = accuracy_score(y_test, y_pred, normalize=False)

    prec = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    return {'accuracy': acc, 
            'precision': prec,
            'recall':recall, 
            'accuracy_count':num_acc}

In [87]:
def build_model(classifier_fn,                
                name_of_y_col, 
                names_of_x_cols, 
                dataset, 
                test_frac=0.2):
    
    X = dataset[names_of_x_cols]
    Y = dataset[name_of_y_col]

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_frac)
       
    model = classifier_fn(x_train, y_train)
    
    y_pred = model.predict(x_test)

    y_pred_train = model.predict(x_train)
    
    train_summary = summarize_classification(y_train, y_pred_train)
    test_summary = summarize_classification(y_test, y_pred)
    
    pred_results = pd.DataFrame({'y_test': y_test,
                                 'y_pred': y_pred})
    
    print(pred_results.sample(10))
    
    model_crosstab = pd.crosstab(pred_results.y_pred, pred_results.y_test)
    
    return {'training': train_summary, 
            'test': test_summary,
            'confusion_matrix': model_crosstab}

In [88]:
def compare_results():
    for key in result_dict:
        print('Classification: ', key)

        print()
        print('Training data')
        for score in result_dict[key]['training']:
            print(score, result_dict[key]['training'][score])

        print()
        print('Test data')
        for score in result_dict[key]['test']:
            print(score, result_dict[key]['test'][score])
       
        print()

In [89]:
def logistic_fn(x_train, y_train):
    
    model = LogisticRegression(solver='liblinear')
    model.fit(x_train, y_train)
    
    return model

In [90]:
def linear_discriminant_fn(x_train, y_train, solver='svd'):
    
    model = LinearDiscriminantAnalysis(solver=solver)
    model.fit(x_train, y_train)
    
    return model

In [91]:
def quadratic_discriminant_fn(x_train, y_train):
    
    model = QuadraticDiscriminantAnalysis()
    model.fit(x_train, y_train)
    
    return model

In [92]:
def sgd_fn(x_train, y_train, max_iter=1000, tol=1e-3):
    
    model = SGDClassifier(max_iter=max_iter, tol=tol)
    model.fit(x_train, y_train)
     
    return model

In [93]:
def linear_svc_fn(x_train, y_train, C=1.0, max_iter=1000, tol=1e-3):
    
    model = LinearSVC(C=C, max_iter=max_iter, tol=tol, dual=False)
    model.fit(x_train, y_train) 
    
    return model

In [94]:
def decision_tree_fn(x_train, y_train, max_depth=None, max_features=None): 
    
    model = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
    model.fit(x_train, y_train)
    
    return model

In [95]:
def naive_bayes_fn(x_train,y_train, priors=None):
    
    model = GaussianNB(priors=priors)
    model.fit(x_train, y_train)
    
    return model

In [107]:
m9= m8.loc[m8['is_goal_x'] == 5]
m9.to_csv('test_data.csv',index=False)

In [108]:
m10= m8.loc[m8['is_goal_x'] != 5]
m10.to_csv('train_data.csv',index=False)

In [109]:
a1 = pd.read_csv("train_data.csv")
a2 = pd.read_csv("test_data.csv")
a2.columns
col = ['shot_id_number', 'power_of_shot', 'knockout_match', 'is_goal_x',
       'home/away', 'distance_of_shot_avg', 'remaining_min_1',
       'area_of_shot_Center(C)', 'area_of_shot_Left Side Center(LC)',
       'area_of_shot_Left Side(L)', 'area_of_shot_Mid Ground(MG)',
       'area_of_shot_Right Side Center(RC)', 'area_of_shot_Right Side(R)',
       'area_of_shot_U', 'shot_basics_Goal Area', 'shot_basics_Goal Line',
       'shot_basics_Left Corner', 'shot_basics_Mid Ground Line',
       'shot_basics_Mid Range', 'shot_basics_Penalty Spot',
       'shot_basics_Right Corner', 'shot_basics_U', 'opposing_team',
       'match_id_n', 'game_season_n', 'type_of_shot_n',
       'type_of_combined_shot_n']
for i in col:
    a2[i] = pd.to_numeric(a2[i])

new_x = a2.drop('is_goal_x',axis=1)

In [110]:
new_x.head()

,shot_id_number,power_of_shot,knockout_match,home/away,distance_of_shot_avg,remaining_min_1,area_of_shot_Center(C),area_of_shot_Left Side Center(LC),area_of_shot_Left Side(L),area_of_shot_Mid Ground(MG),...,shot_basics_Mid Ground Line,shot_basics_Mid Range,shot_basics_Penalty Spot,shot_basics_Right Corner,shot_basics_U,opposing_team,match_id_n,game_season_n,type_of_shot_n,type_of_combined_shot_n


In [103]:
a1.columns

Index(['shot_id_number', 'power_of_shot', 'knockout_match', 'is_goal_x',
       'home/away', 'distance_of_shot_avg', 'remaining_min_1',
       'area_of_shot_Center(C)', 'area_of_shot_Left Side Center(LC)',
       'area_of_shot_Left Side(L)', 'area_of_shot_Mid Ground(MG)',
       'area_of_shot_Right Side Center(RC)', 'area_of_shot_Right Side(R)',
       'area_of_shot_U', 'shot_basics_Goal Area', 'shot_basics_Goal Line',
       'shot_basics_Left Corner', 'shot_basics_Mid Ground Line',
       'shot_basics_Mid Range', 'shot_basics_Penalty Spot',
       'shot_basics_Right Corner', 'shot_basics_U', 'opposing_team',
       'match_id_n', 'game_season_n', 'type_of_shot_n',
       'type_of_combined_shot_n'],
      dtype='object')

In [ ]:
result_dict['survived ~ dt'] = build_model(decision_tree_fn,
                                              'is_goal_x',
                                               [ 'power_of_shot', 'knockout_match',
       'home/away', 'distance_of_shot_avg', 'remaining_min_1',
       'area_of_shot_Center(C)', 'area_of_shot_Left Side Center(LC)',
       'area_of_shot_Left Side(L)', 'area_of_shot_Mid Ground(MG)',
       'area_of_shot_Right Side Center(RC)', 'area_of_shot_Right Side(R)',
       'area_of_shot_U', 'shot_basics_Goal Area', 'shot_basics_Goal Line',
       'shot_basics_Left Corner', 'shot_basics_Mid Ground Line',
       'shot_basics_Mid Range', 'shot_basics_Penalty Spot',
       'shot_basics_Right Corner', 'shot_basics_U', 'opposing_team',
       'match_id_n', 'game_season_n', 'type_of_shot_n',
       'type_of_combined_shot_n'],
                                               a1)

compare_results()

In [ ]:
from sklearn.model_selection import train_test_split

X = a1.drop('is_goal_x', axis=1)
Y = a1['is_goal_x']



x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2565802939)
#x_test = a2.drop('is_goal_x',axis=1)

In [ ]:
a1.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train)

y_pred=clf.predict(new_x)

In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
! pip install --user xgboost

In [ ]:
# First XGBoost model for Pima Indians dataset
import xgboost
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# fit model no training data
model = XGBClassifier()
model.fit(x_train, y_train)
# make predictions for test data
y_pred = model.predict(new_x)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
pred_results = pd.DataFrame({'y_pred': y_pred})
pred_results.to_csv('sub.csv',index=False)
pred_results